Notes from [Introduction to Synthetic Aperture Radar Using Python and MATLAB](https://ieeexplore.ieee.org/document/9893146).

# The Range Doppler Algorithm

The range-Doppler algorithm (RDA) is one of the most widely used image formation algorithms for Synthetic Aperture Radar. It is the primary image formation algorithm used by [GAMMA Software](https://gamma-rs.ch/uploads/media/GAMMA_Software_information.pdf), for example. RDA handles the range and azimuth domains, seperately, as one-dimensional processes. Separately handling the range and azimuth domains is an approximation that leads to errors. These errors must be corrected through *range cell migration correction* (RCMC).

### Algorithm Steps

##### 1. Range Compression
##### 2. Convert to Range-Doppler Domain
##### 3. Range Cell Migration Correction
##### 4. Azimuth Compression

### Range Compression
Range compression is most commonly performed by a simple [match filter](./pulse_compression.ipynb). Attenuation, such as a sensitivity time control attenuator can be added to compensate for amplitude variation due to range. 

### Conversion to Range-Doppler Domain
Range-Doppler domain conversion is done via a one-dimensional fourier transform across the azimuth dimension. The signal from a point target will follow a hyperbolic path in the range-Doppler domain. Because of this, the signal can cross multiple range cells; these range cells must be aligned by using RCMC or the final image will be blurred.

### Range Cell Migration Correction (RCMC)
Range cell migration correction is needed to ensure that range cells are aligned after converting to the range-Doppler domain. Before this can be done, the Doppler Centroid must be accurately estimated. Any errors in the Doppler centroid estimation will translate to poor SNR, ambiguity, among other problems. One technique for Doppler centroid estimation is the average phase change method. The average phase change method finds the mean of the phase change between adjacent azimuth cells for each range cell. This process is given by
$$ f_{DC} = \frac{\sum_{i=1}^{N-1}B_i}{2 \pi N PRF}  \text{\;\;(Hz)}$$
where N is the number of azimuth angles, 
$$ B_i = \angle{\sum_{j=1}^{M} s_{i, j} \times s_{i+1, j}^{*} } \text{\;\;(rad)}$$
M is the number of range cells, and $s_{i, j}$ is the range-Doppler data. Once the Doppler centroid is found, the correction for each Doppler frequency is
$$ R_{offset}^{m, n} = \frac{R_m}{\sqrt{1 - (\frac{\lambda \Delta f n}{2 v_{eff}})^2}} - R_m \text{\;\;(m)}$$
where
$$ \Delta f = \frac{PRF}{N} + f_{DC} \text{\;\;(Hz)}$$

### Azimuth Compression

Once range cell migration correction has been performed, the data needs to be compressed in the azimuth direction via a matched filter. The match filter is build using an azimuth reference function. The azimuth reference function uses the Doppler centroid estimation, and depends on the fact that the Doppler frequency in the azimuth direction is a chirp.<br>

<img src="imgs/doppler_freq.png"  width="500" height="500" style="margin-left:auto; margin-right:auto"/>

The doppler frequency slope in the azimuth direction is given by
$$ \beta_{az} = \frac{2v_{eff}^{2}}{\lambda R_s} \text{\;\;(Hz)}$$
where $R_s$ is the squinted range for each image cell
$$ R_{s}^{m,n} = \frac{R_m}{\sqrt{1 - (\frac{\lambda \Delta f n}{2 v_{eff}}})^2} \text{\;\;(m)}$$
Each range cell requires a unique reference function.